# T/UDOM/2017/04725

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:


# Load the train set and store them into variable named " data "
data= pd.read_csv("C:/users/zai/Desktop/traindata.csv")
data.head(4)

,continue_drop,student_id,gender,caste,mathematics_marks,english_marks,science_marks,science_teacher,languages_teacher,guardian,internet,school_id,total_students,total_toilets,establishment_year
0,continue,s01746,M,BC,0.666,0.468,0.666,7,6,other,True,305,354,86.0,1986.0
1,continue,s16986,M,BC,0.172,0.420,0.172,8,10,mother,False,331,516,15.0,1996.0
2,continue,s00147,F,BC,0.212,0.601,0.212,1,4,mother,False,311,209,14.0,1976.0
3,continue,s08104,F,ST,0.434,0.611,0.434,2,5,father,True,364,147,28.0,1911.0


In [3]:
data.columns

Index(['continue_drop', 'student_id', 'gender', 'caste', 'mathematics_marks',
       'english_marks', 'science_marks', 'science_teacher',
       'languages_teacher', 'guardian', 'internet', 'school_id',
       'total_students', 'total_toilets', 'establishment_year'],
      dtype='object')

In [4]:
data=data.replace({'continue':1,'drop':0})

In [5]:
data['caste'].unique()

array(['BC', 'ST', 'SC', 'OC'], dtype=object)

In [6]:
data=data.replace({'F':1,'M':0})

In [7]:
data=data.replace({'other':0,'father':1,'mother':2,'mixed':3})

In [8]:
data=data.replace({'True':1,'False':0})

In [9]:
data=data.replace({'BC':0,'ST':1,'SC':2,'OC':3})

In [10]:


x=data.corr()
x['continue_drop'].sort_values(ascending=False)



continue_drop         1.000000
caste                 0.038462
guardian              0.033059
languages_teacher     0.018842
science_marks         0.007258
mathematics_marks     0.007258
school_id             0.000396
establishment_year   -0.000519
total_students       -0.005051
total_toilets        -0.009832
gender               -0.043430
internet             -0.060419
english_marks        -0.073242
science_teacher      -0.127864
Name: continue_drop, dtype: float64

In [11]:
data.drop(['student_id'],axis=1,inplace=True)

In [56]:
data.drop(['science_marks','mathematics_marks','school_id','establishment_year','total_students','total_toilets'], axis=1 ,inplace=True)
data.head()

ValueError: labels ['science_marks' 'mathematics_marks' 'school_id' 'establishment_year'
 'total_students' 'total_toilets'] not contained in axis

# 2. Data Cleaning

In [13]:
data.isnull().sum()

continue_drop        0
gender               0
caste                0
english_marks        0
science_teacher      0
languages_teacher    0
guardian             0
internet             0
dtype: int64

# 3. Feature Engineering


i) Feature Selection

In [14]:
target =data['continue_drop'].copy()

In [15]:
target.tail(5)

17185    0
17186    1
17187    1
17188    1
17189    1
Name: continue_drop, dtype: int64

In [16]:
#checking on data imbalance

target.value_counts()

1    16384
0      806
Name: continue_drop, dtype: int64

In [59]:


#visualizing how is data imbalance in column co

import seaborn as sns
plt.figure(figsize=(7,7))
sns.countplot(data=data, x='continue_drop')
plt.title("Data imbalance \n (1 = continue , 0 = Drop) ", fontsize = 15)
plt.xlabel("Continue Vs Drop", fontsize = 15);
plt.ylabel("Total Number of Students", fontsize = 15);
labels = ['Continue','Drop']
val = [1,0] 
plt.xticks(val, labels);



In [18]:


#handling imbalance class
majority = data[data['continue_drop']==1]
minority = data[data['continue_drop']==0]



In [19]:
#check the shape of majority class
majority.shape

(16384, 8)

In [20]:
# check the shape for minority class
minority.shape

(806, 8)

In [21]:


# get class counts 
count_1,count_0 = data['continue_drop'].value_counts()



In [22]:
# upsampling majority class
upsampled = minority.sample(count_1,replace=True)
upsampled.sample(4)

,continue_drop,gender,caste,english_marks,science_teacher,languages_teacher,guardian,internet
249,0,1,0,0.820,4,9,2,True
1169,0,0,2,0.662,4,2,2,True
10972,0,1,0,0.947,6,3,2,True
14187,0,1,0,0.699,6,0,1,True


In [23]:
data_balanced1 = pd.concat([upsampled,majority], axis = 0)

In [24]:
downsampled = majority.sample(count_1,random_state=100)

In [25]:


data_balanced = pd.concat([upsampled,downsampled], axis = 0)



In [26]:

import seaborn as sns
plt.figure(figsize=(4,6))
sns.countplot(data_balanced['continue_drop'])
plt.title("After Data balancing \n (1 = continue , 0 = Drop) ", fontsize = 15)
plt.xlabel("Continue Vs Drop", fontsize = 5);
plt.ylabel("Total Number of Students", fontsize = 5);
labels = ['Continue','Drop']
val = [1,0] 
plt.xticks(val, labels);

In [27]:
data_balanced['continue_drop'].value_counts()

1    16384
0    16384
Name: continue_drop, dtype: int64

In [28]:
features = data.drop(['continue_drop'],axis=1)

In [29]:


features.head()



,gender,caste,english_marks,science_teacher,languages_teacher,guardian,internet
0,0,0,0.468,7,6,0,True
1,0,0,0.420,8,10,2,False
2,1,0,0.601,1,4,2,False
3,1,1,0.611,2,5,1,True
4,1,2,0.478,1,10,2,True


# 4. Training and Evaluation

# i) Data Splitting

In [30]:
import sklearn
from sklearn.model_selection import train_test_split

In [31]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=.30, random_state=42)

# ii) Model selection

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
# defining the model with its associated parameters
from sklearn.neighbors import KNeighborsClassifier       
knn = KNeighborsClassifier(n_neighbors=5)               
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

# iii) Prediction


In [34]:
y_pred = knn.predict(x_test)

# iv) Evalution Perfomance measure

Using the following metrics

    confusion matrix
    f1 score



# Confusion matrix

In [35]:
from sklearn.metrics import confusion_matrix
ans=confusion_matrix(y_test,y_pred)
ans

array([[ 244,    0],
       [   0, 4913]])

In [36]:
from sklearn.metrics import f1_score
f1_acc = f1_score(y_test,y_pred)
f1_acc

1.0

# 5. Testing

In [37]:
data2=pd.read_csv('C:/users/zai/Desktop/testdata.csv')

In [38]:
data2.head(2)

,student_id,gender,caste,mathematics_marks,english_marks,science_marks,science_teacher,languages_teacher,guardian,internet,school_id,total_students,total_toilets,establishment_year
0,s00529,F,BC,0.212,0.601,0.212,1,4,mother,False,341,430,44.0,1959.0
1,s13519,F,BC,0.511,0.551,0.511,1,5,mother,True,329,275,60.0,1914.0


In [39]:
data2['guardian'].unique()

array(['mother', 'father', 'other', 'mixed'], dtype=object)

In [40]:


data2['caste'].unique()



array(['BC', 'SC', 'OC', 'ST'], dtype=object)

In [41]:


data2['gender'].unique()



array(['F', 'M'], dtype=object)

In [42]:
data2['internet'].unique()

array([False,  True], dtype=bool)

In [43]:
data2=data2.replace({True:1,False:0})

In [44]:
data2['internet'].unique()

array([0, 1], dtype=int64)

In [45]:
data2=data2.replace({'BC':0,'ST':1,'SC':2,'OC':3})

In [46]:
data2=data2.replace({'F':1,'M':0})

In [47]:
data2=data2.replace({'other':0,'father':1,'mother':2,'mixed':3})

In [48]:


data2.dtypes



student_id             object
gender                  int64
caste                   int64
mathematics_marks     float64
english_marks         float64
science_marks         float64
science_teacher         int64
languages_teacher       int64
guardian                int64
internet                int64
school_id               int64
total_students          int64
total_toilets         float64
establishment_year    float64
dtype: object

In [49]:
data2.drop(['science_marks','mathematics_marks','school_id','establishment_year','total_students','total_toilets'], axis=1 ,inplace=True)

In [50]:


data2.drop(['student_id'],axis=1,inplace=True)



In [51]:


data2.head()



,gender,caste,english_marks,science_teacher,languages_teacher,guardian,internet
0,1,0,0.601,1,4,2,0
1,1,0,0.551,1,5,2,1
2,0,0,0.644,2,2,2,1
3,1,0,0.373,7,4,2,1
4,1,0,0.147,4,3,1,1


In [52]:
test=knn.predict(data2)

In [53]:
test

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [54]:
test=pd.DataFrame(test,columns=['predictions'])
test.head(3)

,predictions
0,1
1,1
2,1


# 6. Saving the model

In [58]:
test.to_csv('C:/users/zai/Desktop/StudentDropOut.npy')